
# 🧪 Lab 1 — Generative Models Foundations: **GAN vs VAE** (PyTorch, MNIST)

**Course:** Generative AI (Day 1)  
**Lab Length:** ~3 hours  
**Goal:** Implement a **GAN** and a **VAE** on MNISTFashion-MNIST, compare their behavior, and reflect on stability vs. latent structure.

> ✅ **What you will submit:**  
> - A short reflection with generated image grids and comments.

### 🚦 Rules
- Keep training epochs small if you’re on CPU; you can always re-run with more epochs later.
- Cells marked **(Provided)** can be run as-is; cells marked **(TODO)** require your edits.

### 🧰 Requirements
- PyTorch, Torchvision, Matplotlib, TQDM
- (Optional) SciPy for a simple FID-like metric



## 🎯 Learning Objectives
- Implement a **vanilla GAN**: Generator + Discriminator, adversarial loss, and training loop.
- Implement a **Variational Autoencoder (VAE)**: encoder/decoder, **reparameterization trick**, and **ELBO**.
- Produce **visualizations**: sample grids, reconstructions, and **latent interpolations**.
- Compare GAN vs VAE using a **proxy FID-like** feature distance.
- Reflect on stability, mode collapse, and smoothness of latent space.



## 🧭 Tips for Success
- Use **[-1, 1]** input range for GANs (Tanh output).  
- Start with **small networks**. You can always scale up.  
- Inspect **loss curves**, generated images, and reconstructions **frequently**.
- If your GAN collapses, try: smaller LR, label smoothing, or tweak BatchNorm.


In [ ]:

# ===== (Provided) Setup & Installs =====
# If you're in Colab, uncomment the next line to ensure dependencies:
# !pip -q install torch torchvision torchaudio matplotlib tqdm scipy

import math, os, random, numpy as np, torch, torch.nn as nn, torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, utils
import matplotlib.pyplot as plt
from tqdm import tqdm

SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED); torch.cuda.manual_seed_all(SEED)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)


: 

In [ ]:

# ===== (Provided) Helper: visualization & Results Saving =====
import os
from datetime import datetime

# Create results directory
RESULTS_DIR = "results"
os.makedirs(RESULTS_DIR, exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/models", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/gan_samples", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/vae_reconstructions", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/vae_samples", exist_ok=True)
os.makedirs(f"{RESULTS_DIR}/vae_interpolations", exist_ok=True)

def show_grid(tensor, title="", nrow=4, value_range=(-1,1), save_path=None):
    grid = utils.make_grid(tensor, nrow=nrow, normalize=True, value_range=value_range)
    plt.figure(figsize=(6,6)); plt.axis('off'); plt.title(title, fontsize=12)
    plt.imshow(grid.permute(1,2,0)); 
    if save_path:
        plt.savefig(save_path, bbox_inches='tight', dpi=150)
        print(f"Saved: {save_path}")
    plt.show()

# Mode collapse detection: check diversity in generated samples
def detect_mode_collapse(samples, threshold=0.1):
    """
    Detect mode collapse by checking if samples are too similar.
    Returns: (is_collapsed, diversity_score)
    """
    # Check diversity: compute pairwise L2 distances between samples
    samples_flat = samples.view(samples.size(0), -1)
    # Normalize
    samples_norm = F.normalize(samples_flat, p=2, dim=1)
    # Compute pairwise cosine similarities
    similarity_matrix = torch.mm(samples_norm, samples_norm.t())
    # Average similarity (excluding diagonal)
    avg_similarity = (similarity_matrix.sum() - similarity_matrix.trace()) / (len(samples) * (len(samples) - 1))
    diversity_score = 1.0 - avg_similarity.item()
    
    # Mode collapse if samples are too similar
    is_collapsed = diversity_score < threshold
    
    return is_collapsed, diversity_score

# Storage for all experiment results
all_results = {
    'gan': {},
    'vae': {}
}


In [ ]:

# ===== Data Loading for Both Datasets =====
BATCH = 128

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load both datasets
datasets_dict = {}
for use_fashion, dataset_name in [(False, 'MNIST'), (True, 'FashionMNIST')]:
    if use_fashion:
        train_ds = datasets.FashionMNIST(root='./data', train=True, download=True, transform=transform)
        test_ds  = datasets.FashionMNIST(root='./data', train=False, download=True, transform=transform)
    else:
        train_ds = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
        test_ds  = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
    
    train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2, pin_memory=True)
    test_loader  = DataLoader(test_ds, batch_size=BATCH, shuffle=False, num_workers=2, pin_memory=True)
    
    datasets_dict[dataset_name] = {
        'train_loader': train_loader,
        'test_loader': test_loader,
        'train_ds': train_ds,
        'test_ds': test_ds
    }
    
    # Quick sanity-check visualization
    xb, yb = next(iter(train_loader))
    show_grid(xb[:16], title=f"{dataset_name} Real samples (normalized to [-1,1])", save_path=None)

print(f"✓ Loaded datasets: {list(datasets_dict.keys())}")



---

# Part 1 — **GAN** (Vanilla)  *(~90 minutes)*

We will implement a simple GAN (DCGAN-ish) with:
- **Generator**: maps `z ~ N(0, I)` to image `x̂`
- **Discriminator**: scores real vs fake
- **Loss**: Hinge (recommended) **or** BCE (your choice)

> **Milestones**
> 1) Implement **Generator** & **Discriminator**  
> 2) Choose **loss** (hinge recommended), set **optimizers**  
> 3) Implement **training loop** and generate sample grids every epoch


In [ ]:

# ===== (TODO) GAN Architectures =====
IMG_CH = 1
IMG_H  = 28
IMG_W  = 28

class Generator(nn.Module):
    def __init__(self, z_dim=64, img_ch=IMG_CH):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(z_dim, 128*7*7),
            nn.BatchNorm1d(128*7*7),
            nn.ReLU(True),
            nn.Unflatten(1, (128, 7, 7)),
            nn.ConvTranspose2d(128, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.Conv2d(32, img_ch, 3, 1, 1),
            nn.Tanh()
        )
    def forward(self, z):
        return self.net(z)

class Discriminator(nn.Module):
    def __init__(self, img_ch=IMG_CH):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(img_ch, 32, 4, 2, 1),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(32, 64, 4, 2, 1),
            nn.BatchNorm2d(64),
            nn.LeakyReLU(0.2, True),
            nn.Conv2d(64, 128, 3, 2, 1),
            nn.LeakyReLU(0.2, True),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*4*4, 1)
        )
    def forward(self, x):
        f = self.features(x)
        logits = self.classifier(f).squeeze(1)
        return logits, f

print("✓ GAN architectures defined.")


In [ ]:

# ===== (TODO) GAN Losses =====
# Option A: Hinge loss (recommended)
def d_loss_hinge(real_logits, fake_logits):
    return F.relu(1.0 - real_logits).mean() + F.relu(1.0 + fake_logits).mean()

def g_loss_hinge(fake_logits):
    return -fake_logits.mean()

# Option B: BCE 
bce = nn.BCEWithLogitsLoss()
def d_loss_bce(real_logits, fake_logits):
    real_t = torch.ones_like(real_logits)
    fake_t = torch.zeros_like(fake_logits)
    return bce(real_logits, real_t) + bce(fake_logits, fake_t)

def g_loss_bce(fake_logits):
    real_t = torch.ones_like(fake_logits)
    return bce(fake_logits, real_t)


In [ ]:

# ===== GAN Training - All Experiments for Both Datasets =====
# Run all combinations: Dataset × Loss Type × Z_DIM

LR = 2e-4
betas = (0.5, 0.999)
EPOCHS_GAN = 10

# Experiment configurations
gan_configs = [
    {'loss_type': 'hinge', 'z_dim': 64},
    {'loss_type': 'hinge', 'z_dim': 32},
    {'loss_type': 'hinge', 'z_dim': 128},
    {'loss_type': 'bce', 'z_dim': 64},
    {'loss_type': 'bce', 'z_dim': 32},
    {'loss_type': 'bce', 'z_dim': 128},
]

# Run experiments for each dataset
for dataset_name, data_loaders in datasets_dict.items():
    train_loader = data_loaders['train_loader']
    test_loader = data_loaders['test_loader']
    
    print(f"\n{'#'*70}")
    print(f"# DATASET: {dataset_name}")
    print(f"{'#'*70}\n")
    
    for config in gan_configs:
        loss_type = config['loss_type']
        z_dim = config['z_dim']
        exp_name = f"{dataset_name}_GAN_{loss_type}_z{z_dim}"
        print(f"\n{'='*60}")
        print(f"Training: {exp_name}")
        print(f"{'='*60}")
        
        # Initialize models
        G = Generator(z_dim=z_dim).to(device)
        D = Discriminator().to(device)
        opt_G = torch.optim.Adam(G.parameters(), lr=LR, betas=betas)
        opt_D = torch.optim.Adam(D.parameters(), lr=LR, betas=betas)
        
        # Select loss functions
        if loss_type == 'hinge':
            d_loss_fn = d_loss_hinge
            g_loss_fn = g_loss_hinge
        else:
            d_loss_fn = d_loss_bce
            g_loss_fn = g_loss_bce
        
        # Fixed noise for consistent visualization
        fixed_z = torch.randn(16, z_dim, device=device)
        
        # Store results
        all_results['gan'][exp_name] = {
            'dataset': dataset_name,
            'loss_type': loss_type,
            'z_dim': z_dim,
            'losses_d': [],
            'losses_g': [],
            'best_epoch': 0,
            'best_samples': None,
            'mode_collapse_epochs': [],
            'diversity_scores': []
        }
        
        for epoch in range(1, EPOCHS_GAN+1):
            G.train(); D.train()
            epoch_losses_d = []
            epoch_losses_g = []
            
            pbar = tqdm(train_loader, desc=f"[{exp_name}] Epoch {epoch}/{EPOCHS_GAN}")
            for x, _ in pbar:
                x = x.to(device)

                # (1) Update D
                z = torch.randn(x.size(0), z_dim, device=device)
                with torch.no_grad():
                    x_fake = G(z)
                real_logits, _ = D(x)
                fake_logits, _ = D(x_fake)
                loss_D = d_loss_fn(real_logits, fake_logits)
                opt_D.zero_grad(set_to_none=True)
                loss_D.backward()
                opt_D.step()

                # (2) Update G
                z = torch.randn(x.size(0), z_dim, device=device)
                x_fake = G(z)
                fake_logits, _ = D(x_fake)
                loss_G = g_loss_fn(fake_logits)
                opt_G.zero_grad(set_to_none=True)
                loss_G.backward()
                opt_G.step()

                epoch_losses_d.append(loss_D.item())
                epoch_losses_g.append(loss_G.item())
                pbar.set_postfix({'D': f"{loss_D.item():.3f}", 'G': f"{loss_G.item():.3f}"})

            # Save epoch losses
            avg_loss_d = np.mean(epoch_losses_d)
            avg_loss_g = np.mean(epoch_losses_g)
            all_results['gan'][exp_name]['losses_d'].append(avg_loss_d)
            all_results['gan'][exp_name]['losses_g'].append(avg_loss_g)
            
            # Generate and check samples
            with torch.no_grad():
                samples = G(fixed_z).cpu()
            
            # Check for mode collapse
            is_collapsed, diversity_score = detect_mode_collapse(samples)
            all_results['gan'][exp_name]['diversity_scores'].append(diversity_score)
            
            if is_collapsed:
                all_results['gan'][exp_name]['mode_collapse_epochs'].append(epoch)
                print(f"  ⚠️  Mode collapse detected at epoch {epoch} (diversity: {diversity_score:.3f})")
            
            save_path = f"{RESULTS_DIR}/gan_samples/{exp_name}_epoch{epoch}.png"
            title = f"{exp_name} - Epoch {epoch}"
            if is_collapsed:
                title += " [MODE COLLAPSE]"
            show_grid(samples, title=title, save_path=save_path)
            
            # Save best samples (from last epoch)
            if epoch == EPOCHS_GAN:
                all_results['gan'][exp_name]['best_samples'] = samples
                all_results['gan'][exp_name]['best_epoch'] = epoch
                
                # Save models for FID computation
                torch.save({
                    'G': G.state_dict(),
                    'D': D.state_dict(),
                    'z_dim': z_dim,
                    'loss_type': loss_type
                }, f"{RESULTS_DIR}/models/{exp_name}_final.pth")
        
        print(f"✓ Completed: {exp_name}")
        if all_results['gan'][exp_name]['mode_collapse_epochs']:
            print(f"  Mode collapse detected in epochs: {all_results['gan'][exp_name]['mode_collapse_epochs']}")
        print()

print(f"\n{'='*60}")
print("All GAN experiments completed for all datasets!")
print(f"{'='*60}")



---

# Part 2 — **VAE**  *(~60 minutes)*

We will implement:
- Encoder that outputs **μ** and **log σ²**
- **Reparameterization trick**: `z = μ + σ ⊙ ε`
- Decoder that reconstructs `x̂`
- **ELBO** loss = reconstruction + KL divergence

> **Milestones**
> 1) Build **VAE module** (encode/reparameterize/decode)  
> 2) Implement **loss** (reconstruction + KL)  
> 3) Train and visualize reconstructions & random samples  
> 4) Do a **latent interpolation** between two test images


In [ ]:

# ===== (TODO) VAE Architecture =====
class VAE(nn.Module):
    def __init__(self, latent=16):
        super().__init__()
        # Encoder
        self.enc = nn.Sequential(
            nn.Conv2d(1, 32, 4, 2, 1),   # 14x14
            nn.ReLU(True),
            nn.Conv2d(32, 64, 4, 2, 1),  # 7x7
            nn.ReLU(True),
            nn.Flatten()
        )
        self.enc_fc_mu  = nn.Linear(64*7*7, latent)
        self.enc_fc_log = nn.Linear(64*7*7, latent)

        # Decoder
        self.dec_fc = nn.Linear(latent, 64*7*7)
        self.dec = nn.Sequential(
            nn.Unflatten(1, (64, 7, 7)),
            nn.ConvTranspose2d(64, 32, 4, 2, 1),  # 14x14
            nn.ReLU(True),
            nn.ConvTranspose2d(32, 16, 4, 2, 1),  # 28x28
            nn.ReLU(True),
            nn.Conv2d(16, 1, 3, 1, 1),
            nn.Tanh()
        )

    def encode(self, x):
        h = self.enc(x)
        mu = self.enc_fc_mu(h)
        logvar = self.enc_fc_log(h)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = (0.5*logvar).exp()
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = self.dec_fc(z)
        x = self.dec(h)
        return x

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        xhat = self.decode(z)
        return xhat, mu, logvar

print("✓ VAE architecture defined.")


In [ ]:

# ===== VAE Loss & Training - All Experiments for Both Datasets =====
# Run all latent dimensions: [16, 8, 32] for both datasets

def vae_loss(xhat, x, mu, logvar):
    recon = F.l1_loss(xhat, x, reduction='sum') / x.size(0)
    kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)
    return recon + kl, recon, kl

EPOCHS_VAE = 10
vae_latent_dims = [16, 8, 32]

# Run experiments for each dataset
for dataset_name, data_loaders in datasets_dict.items():
    train_loader = data_loaders['train_loader']
    test_loader = data_loaders['test_loader']
    
    print(f"\n{'#'*70}")
    print(f"# DATASET: {dataset_name} - VAE Experiments")
    print(f"{'#'*70}\n")
    
    # Get fixed test samples for reconstruction visualization
    fixed_test_samples, _ = next(iter(test_loader))
    fixed_test_samples = fixed_test_samples[:16].to(device)
    
    for latent_dim in vae_latent_dims:
        exp_name = f"{dataset_name}_VAE_latent{latent_dim}"
        print(f"\n{'='*60}")
        print(f"Training: {exp_name}")
        print(f"{'='*60}")
        
        # Initialize VAE
        vae = VAE(latent=latent_dim).to(device)
        opt_vae = torch.optim.Adam(vae.parameters(), lr=2e-3)
        
        # Store results
        all_results['vae'][exp_name] = {
            'dataset': dataset_name,
            'latent_dim': latent_dim,
            'losses': [],
            'recon_losses': [],
            'kl_losses': [],
            'best_epoch': 0,
            'best_reconstructions': None,
            'random_samples': None,
            'interpolation': None
        }
        
        for epoch in range(1, EPOCHS_VAE+1):
            vae.train()
            epoch_losses = []
            epoch_recon = []
            epoch_kl = []
            
            pbar = tqdm(train_loader, desc=f"[{exp_name}] Epoch {epoch}/{EPOCHS_VAE}")
            for x, _ in pbar:
                x = x.to(device)
                xhat, mu, logvar = vae(x)
                loss, rec, kl = vae_loss(xhat, x, mu, logvar)
                opt_vae.zero_grad(set_to_none=True)
                loss.backward()
                opt_vae.step()
                epoch_losses.append(loss.item())
                epoch_recon.append(rec.item())
                epoch_kl.append(kl.item())
                pbar.set_postfix({'loss': f"{np.mean(epoch_losses):.2f}", 
                                'recon': f"{np.mean(epoch_recon):.2f}", 
                                'kl': f"{np.mean(epoch_kl):.2f}"})
            
            # Save epoch losses
            all_results['vae'][exp_name]['losses'].append(np.mean(epoch_losses))
            all_results['vae'][exp_name]['recon_losses'].append(np.mean(epoch_recon))
            all_results['vae'][exp_name]['kl_losses'].append(np.mean(epoch_kl))
            
            # Visualize reconstructions
            vae.eval()
            with torch.no_grad():
                xhat, _, _ = vae(fixed_test_samples)
            
            save_path_recon = f"{RESULTS_DIR}/vae_reconstructions/{exp_name}_epoch{epoch}.png"
            show_grid(fixed_test_samples.cpu(), title=f"{exp_name} Inputs - Epoch {epoch}", save_path=None)
            show_grid(xhat.cpu(), title=f"{exp_name} Reconstructions - Epoch {epoch}", save_path=save_path_recon)
            
            # Save best reconstructions
            if epoch == EPOCHS_VAE:
                all_results['vae'][exp_name]['best_reconstructions'] = xhat.cpu()
                all_results['vae'][exp_name]['best_epoch'] = epoch
                
                # Save model for FID computation
                torch.save({
                    'vae': vae.state_dict(),
                    'latent_dim': latent_dim
                }, f"{RESULTS_DIR}/models/{exp_name}_final.pth")
        
        # Generate random samples
        vae.eval()
        with torch.no_grad():
            z = torch.randn(16, latent_dim, device=device)
            random_samples = vae.decode(z).cpu()
        
        save_path_samples = f"{RESULTS_DIR}/vae_samples/{exp_name}_random.png"
        show_grid(random_samples, title=f"{exp_name} Random Samples", save_path=save_path_samples)
        all_results['vae'][exp_name]['random_samples'] = random_samples
        
        # Latent interpolation
        def interpolate(a, b, steps=16):
            alphas = torch.linspace(0, 1, steps, device=a.device).view(-1,1)
            return (1-alphas)*a + alphas*b
        
        with torch.no_grad():
            x, _ = next(iter(test_loader))
            x = x.to(device)[:2]
            mu, logvar = vae.encode(x)
            z1 = mu[0]; z2 = mu[1]
            z_traj = interpolate(z1, z2, steps=16)
            interp_imgs = vae.decode(z_traj).cpu()
        
        save_path_interp = f"{RESULTS_DIR}/vae_interpolations/{exp_name}_interpolation.png"
        show_grid(interp_imgs, title=f"{exp_name} Latent Interpolation", save_path=save_path_interp)
        all_results['vae'][exp_name]['interpolation'] = interp_imgs
        
        print(f"✓ Completed: {exp_name}\n")

print(f"\n{'='*60}")
print("All VAE experiments completed for all datasets!")
print(f"{'='*60}")


In [ ]:

# ===== Sampling & Interpolation already done in training loop above =====
# All VAE outputs (random samples, reconstructions, interpolations) 
# have been generated and saved during training
print("VAE sampling and interpolation completed during training.")



---

# Part 3 — **Comparison & Proxy FID-like Metric**  *(~30 minutes)*

We will compare samples from the GAN and VAE using a **feature Fréchet distance** proxy:
1) Extract features from the **Discriminator** (penultimate conv layer)
2) Fit Gaussians to **real** vs **fake** features
3) Compute **Fréchet distance**:  
   $\|\mu_r-\mu_f\|^2 + \mathrm{Tr}(\Sigma_r + \Sigma_f - 2(\Sigma_r \Sigma_f)^{1/2})$




> This is not the official FID (which uses Inception), but behaves similarly for quick lab work.


In [ ]:

# ===== Proxy FID-like Metric for All Experiments =====
try:
    from scipy import linalg
    SCIPY_OK = True
except Exception:
    SCIPY_OK = False
    print("SciPy not available — skipping proxy FID. You can !pip install scipy and re-run.")

def get_features(disc, loader, n_batches=50, use_fake=False, generator=None, z_dim=64, vae=None, latent_dim=16):
    disc.eval()
    feats = []
    with torch.no_grad():
        for i, (x, _) in enumerate(loader):
            if i >= n_batches: break
            x = x.to(device)
            if use_fake:
                if generator is not None:
                    z = torch.randn(x.size(0), z_dim, device=device)
                    x = generator(z)
                elif vae is not None:
                    z = torch.randn(x.size(0), latent_dim, device=device)
                    x = vae.decode(z)
            _, f = disc(x)
            f = F.adaptive_avg_pool2d(f, 1).flatten(1)  # (B, C)
            feats.append(f.cpu())
    return torch.cat(feats, dim=0).numpy()

def gaussian_stats(X):
    mu = X.mean(axis=0)
    sigma = np.cov(X, rowvar=False)
    return mu, sigma

def frechet_distance(mu1, sigma1, mu2, sigma2, eps=1e-6):
    diff = mu1 - mu2
    covmean, _ = linalg.sqrtm(sigma1.dot(sigma2), disp=False)
    if not np.isfinite(covmean).all():
        covmean = linalg.sqrtm((sigma1 + np.eye(sigma1.shape[0])*eps).dot(sigma2 + np.eye(sigma2.shape[0])*eps))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2*covmean)
    return float(fid)

if SCIPY_OK:
    print("\nComputing Proxy FID scores for all experiments...")
    
    # Compute FID for each dataset separately
    for dataset_name, data_loaders in datasets_dict.items():
        test_loader = data_loaders['test_loader']
        
        print(f"\n{'='*60}")
        print(f"Computing FID for {dataset_name}")
        print(f"{'='*60}")
        
        # Use a trained discriminator from one of the GAN experiments for this dataset
        # Find the first GAN experiment for this dataset to use its discriminator
        reference_disc = None
        for exp_name in all_results['gan'].keys():
            if all_results['gan'][exp_name]['dataset'] == dataset_name:
                model_path = f"{RESULTS_DIR}/models/{exp_name}_final.pth"
                if os.path.exists(model_path):
                    checkpoint = torch.load(model_path, map_location=device)
                    reference_disc = Discriminator().to(device)
                    reference_disc.load_state_dict(checkpoint['D'])
                    reference_disc.eval()
                    print(f"Using discriminator from {exp_name} for feature extraction")
                    break
        
        if reference_disc is None:
            print(f"  Warning: No trained discriminator found for {dataset_name}, skipping FID")
            continue
        
        # Get real features for this dataset using trained discriminator
        print("Extracting real image features...")
        real_feats = get_features(reference_disc, test_loader, n_batches=80, use_fake=False)
        mu_r, sig_r = gaussian_stats(real_feats)
        print(f"  Real features shape: {real_feats.shape}, mean: {mu_r.shape}")
        
        # Compute FID for GAN experiments of this dataset
        print("\nComputing FID for GAN experiments...")
        for exp_name in all_results['gan'].keys():
            if all_results['gan'][exp_name]['dataset'] != dataset_name:
                continue
                
            # Load saved model
            model_path = f"{RESULTS_DIR}/models/{exp_name}_final.pth"
            if os.path.exists(model_path):
                checkpoint = torch.load(model_path, map_location=device)
                z_dim = checkpoint['z_dim']
                
                G_temp = Generator(z_dim=z_dim).to(device)
                G_temp.load_state_dict(checkpoint['G'])
                G_temp.eval()
                
                # Use the same trained discriminator for feature extraction
                fake_feats = get_features(reference_disc, test_loader, n_batches=80, use_fake=True, 
                                        generator=G_temp, z_dim=z_dim)
                mu_f, sig_f = gaussian_stats(fake_feats)
                fid_score = frechet_distance(mu_r, sig_r, mu_f, sig_f)
                
                all_results['gan'][exp_name]['fid'] = fid_score
                print(f"  {exp_name}: FID = {fid_score:.2f}")
            else:
                print(f"  {exp_name}: Model file not found, skipping FID")
        
        # Compute FID for VAE experiments of this dataset
        print("\nComputing FID for VAE experiments...")
        for exp_name in all_results['vae'].keys():
            if all_results['vae'][exp_name]['dataset'] != dataset_name:
                continue
                
            # Load saved model
            model_path = f"{RESULTS_DIR}/models/{exp_name}_final.pth"
            if os.path.exists(model_path):
                checkpoint = torch.load(model_path, map_location=device)
                latent_dim = checkpoint['latent_dim']
                
                vae_temp = VAE(latent=latent_dim).to(device)
                vae_temp.load_state_dict(checkpoint['vae'])
                vae_temp.eval()
                
                # Use the same trained discriminator for feature extraction
                fake_feats = get_features(reference_disc, test_loader, n_batches=80, use_fake=True, 
                                        vae=vae_temp, latent_dim=latent_dim)
                mu_f, sig_f = gaussian_stats(fake_feats)
                fid_score = frechet_distance(mu_r, sig_r, mu_f, sig_f)
                
                all_results['vae'][exp_name]['fid'] = fid_score
                print(f"  {exp_name}: FID = {fid_score:.2f}")
            else:
                print(f"  {exp_name}: Model file not found, skipping FID")
    
    print(f"\n{'='*60}")
    print("FID computation completed!")
    print(f"{'='*60}")
else:
    print("SciPy not available. Skipping FID computation.")



---

## 📝 Final Reflection (to submit)

1. Copy all the generated outputs, don't forget to label them (e.g   Fashion-MNIST, GAN, Z_DIM=128, EPOCH=... )

2. Include image grids:
   - GAN samples (best epoch)
   - VAE reconstructions
   - VAE latent interpolation

3. Include your **proxy FID-like** numbers for GAN and VAE.

4. Answer briefly:
   - What hyperparameters most influenced **GAN stability** in your runs?
   - Evidence of **mode collapse** (if any)? What helped?
   - How did **latent dim** affect VAE reconstructions and samples?
   - One idea to combine benefits of both models (e.g., **VAE-GAN**).


In [ ]:
# ===== Generate Comprehensive Report =====
from datetime import datetime

report_path = f"{RESULTS_DIR}/experiment_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"

with open(report_path, 'w') as f:
    f.write("# 🧪 Lab 1 — Generative Models: GAN vs VAE - Complete Experiment Report\n\n")
    f.write(f"**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write("---\n\n")
    
    # Summary Statistics
    f.write("## Experiment Summary\n\n")
    f.write(f"- **Datasets:** MNIST, FashionMNIST\n")
    f.write(f"- **Total GAN Experiments:** {len(all_results['gan'])}\n")
    f.write(f"- **Total VAE Experiments:** {len(all_results['vae'])}\n")
    f.write(f"- **Epochs per Experiment:** 10\n\n")
    f.write("---\n\n")
    
    # GAN Results by Dataset
    for dataset_name in ['MNIST', 'FashionMNIST']:
        f.write(f"## Part 1: GAN Experiments - {dataset_name}\n\n")
        f.write("### Experiment Configurations\n")
        f.write("- **Loss Types:** Hinge, BCE\n")
        f.write("- **Z Dimensions:** 64, 32, 128\n")
        f.write("- **Total Experiments:** 6\n\n")
        
        f.write("### Results Summary\n\n")
        for exp_name, results in all_results['gan'].items():
            if results.get('dataset') != dataset_name:
                continue
            f.write(f"#### {exp_name}\n")
            f.write(f"- **Loss Type:** {results.get('loss_type', 'N/A')}\n")
            f.write(f"- **Z Dimension:** {results.get('z_dim', 'N/A')}\n")
            f.write(f"- **Best Epoch:** {results['best_epoch']}\n")
            if results['losses_d']:
                f.write(f"- **Final D Loss:** {results['losses_d'][-1]:.4f}\n")
                f.write(f"- **Final G Loss:** {results['losses_g'][-1]:.4f}\n")
            if 'fid' in results:
                f.write(f"- **Proxy FID Score:** {results['fid']:.2f}\n")
            if results.get('mode_collapse_epochs'):
                f.write(f"- **⚠️ Mode Collapse Detected in Epochs:** {results['mode_collapse_epochs']}\n")
                if results.get('diversity_scores'):
                    f.write(f"- **Final Diversity Score:** {results['diversity_scores'][-1]:.4f}\n")
            f.write(f"- **Sample Images:** `results/gan_samples/{exp_name}_epoch*.png`\n\n")
    
    # VAE Results by Dataset
    for dataset_name in ['MNIST', 'FashionMNIST']:
        f.write(f"## Part 2: VAE Experiments - {dataset_name}\n\n")
        f.write("### Experiment Configurations\n")
        f.write("- **Latent Dimensions:** 16, 8, 32\n")
        f.write("- **Total Experiments:** 3\n\n")
        
        f.write("### Results Summary\n\n")
        for exp_name, results in all_results['vae'].items():
            if results.get('dataset') != dataset_name:
                continue
            f.write(f"#### {exp_name}\n")
            f.write(f"- **Latent Dimension:** {results.get('latent_dim', 'N/A')}\n")
            f.write(f"- **Best Epoch:** {results['best_epoch']}\n")
            if results['losses']:
                f.write(f"- **Final Total Loss:** {results['losses'][-1]:.4f}\n")
                f.write(f"- **Final Recon Loss:** {results['recon_losses'][-1]:.4f}\n")
                f.write(f"- **Final KL Loss:** {results['kl_losses'][-1]:.4f}\n")
            if 'fid' in results:
                f.write(f"- **Proxy FID Score:** {results['fid']:.2f}\n")
            f.write(f"- **Reconstruction Images:** `results/vae_reconstructions/{exp_name}_epoch*.png`\n")
            f.write(f"- **Random Samples:** `results/vae_samples/{exp_name}_random.png`\n")
            f.write(f"- **Interpolations:** `results/vae_interpolations/{exp_name}_interpolation.png`\n\n")
    
    # Image Grid Summary with Embedded Images
    f.write("## Generated Images\n\n")
    f.write("All generated images are saved in the `results/` directory. Key images are embedded below:\n\n")
    
    # GAN Best Samples
    f.write("### GAN Generated Samples (Best Epoch)\n\n")
    for dataset_name in ['MNIST', 'FashionMNIST']:
        f.write(f"#### {dataset_name}\n\n")
        for exp_name, results in all_results['gan'].items():
            if results.get('dataset') != dataset_name:
                continue
            best_epoch = results.get('best_epoch', 10)
            img_path = f"{RESULTS_DIR}/gan_samples/{exp_name}_epoch{best_epoch}.png"
            # Use relative path from report location (report is in RESULTS_DIR)
            rel_img_path = f"gan_samples/{exp_name}_epoch{best_epoch}.png"
            if os.path.exists(img_path):
                f.write(f"**{exp_name}** (Epoch {best_epoch}):\n\n")
                f.write(f"![{exp_name}]({rel_img_path})\n\n")
    
    # VAE Reconstructions
    f.write("### VAE Reconstructions (Best Epoch)\n\n")
    for dataset_name in ['MNIST', 'FashionMNIST']:
        f.write(f"#### {dataset_name}\n\n")
        for exp_name, results in all_results['vae'].items():
            if results.get('dataset') != dataset_name:
                continue
            best_epoch = results.get('best_epoch', 10)
            img_path = f"{RESULTS_DIR}/vae_reconstructions/{exp_name}_epoch{best_epoch}.png"
            rel_img_path = f"vae_reconstructions/{exp_name}_epoch{best_epoch}.png"
            if os.path.exists(img_path):
                f.write(f"**{exp_name}** (Epoch {best_epoch}):\n\n")
                f.write(f"![{exp_name} Reconstructions]({rel_img_path})\n\n")
    
    # VAE Random Samples
    f.write("### VAE Random Samples\n\n")
    for dataset_name in ['MNIST', 'FashionMNIST']:
        f.write(f"#### {dataset_name}\n\n")
        for exp_name, results in all_results['vae'].items():
            if results.get('dataset') != dataset_name:
                continue
            img_path = f"{RESULTS_DIR}/vae_samples/{exp_name}_random.png"
            rel_img_path = f"vae_samples/{exp_name}_random.png"
            if os.path.exists(img_path):
                f.write(f"**{exp_name}**:\n\n")
                f.write(f"![{exp_name} Random Samples]({rel_img_path})\n\n")
    
    # VAE Interpolations
    f.write("### VAE Latent Interpolations\n\n")
    for dataset_name in ['MNIST', 'FashionMNIST']:
        f.write(f"#### {dataset_name}\n\n")
        for exp_name, results in all_results['vae'].items():
            if results.get('dataset') != dataset_name:
                continue
            img_path = f"{RESULTS_DIR}/vae_interpolations/{exp_name}_interpolation.png"
            rel_img_path = f"vae_interpolations/{exp_name}_interpolation.png"
            if os.path.exists(img_path):
                f.write(f"**{exp_name}**:\n\n")
                f.write(f"![{exp_name} Interpolation]({rel_img_path})\n\n")
    
    f.write("---\n\n")
    f.write("### All Image Files\n\n")
    f.write("All generated images are saved in the `results/` directory:\n\n")
    f.write("- `gan_samples/` - GAN generated samples for each experiment and epoch\n")
    f.write("- `vae_reconstructions/` - VAE reconstruction comparisons\n")
    f.write("- `vae_samples/` - VAE random samples from latent space\n")
    f.write("- `vae_interpolations/` - VAE latent space interpolations\n")
    f.write("- `models/` - Saved model checkpoints for FID computation\n\n")
    
    # Mode Collapse Analysis
    f.write("## Mode Collapse Analysis\n\n")
    f.write("> **Note:** You can answer all reflection questions regardless of whether mode collapse occurs.\n")
    f.write("> - If mode collapse occurs: analyze which configurations caused it and what helped.\n")
    f.write("> - If mode collapse doesn't occur: note which configurations were stable and why.\n\n")
    
    f.write("### GAN Mode Collapse Summary\n\n")
    mode_collapse_found = False
    for exp_name, results in all_results['gan'].items():
        if results.get('mode_collapse_epochs'):
            mode_collapse_found = True
            f.write(f"- **{exp_name}**: Mode collapse in epochs {results['mode_collapse_epochs']}\n")
            if results.get('diversity_scores'):
                f.write(f"  - Diversity scores: {[f'{s:.3f}' for s in results['diversity_scores']]}\n")
    if not mode_collapse_found:
        f.write("- **No mode collapse detected** in any GAN experiments.\n")
        f.write("- All experiments maintained good diversity throughout training.\n")
        f.write("- This suggests the hyperparameters (loss type, Z_DIM, learning rate) were well-tuned.\n")
    f.write("\n### Stable GAN Configurations (No Mode Collapse)\n\n")
    for exp_name, results in all_results['gan'].items():
        if not results.get('mode_collapse_epochs'):
            f.write(f"- **{exp_name}** (Loss: {results.get('loss_type', 'N/A')}, Z_DIM: {results.get('z_dim', 'N/A')})\n")
            if results.get('diversity_scores'):
                f.write(f"  - Final diversity score: {results['diversity_scores'][-1]:.4f}\n")
    f.write("\n")
    
    # FID Scores Summary
    f.write("## Proxy FID Scores Summary\n\n")
    f.write("### GAN FID Scores\n\n")
    for exp_name, results in all_results['gan'].items():
        if 'fid' in results:
            f.write(f"- **{exp_name}**: {results['fid']:.2f}\n")
    f.write("\n### VAE FID Scores\n\n")
    for exp_name, results in all_results['vae'].items():
        if 'fid' in results:
            f.write(f"- **{exp_name}**: {results['fid']:.2f}\n")
    f.write("\n")
    
    # Reflection Questions with Data-Driven Answers
    f.write("## Reflection Questions & Data-Driven Answers\n\n")
    
    f.write("### 1. What hyperparameters most influenced GAN stability in your runs?\n\n")
    f.write("**Analysis Guide:**\n")
    f.write("- Compare loss curves (D loss and G loss) across different configurations\n")
    f.write("- Compare FID scores: lower FID = better quality\n")
    f.write("- Compare diversity scores: higher = more diverse samples\n")
    f.write("- Look for patterns:\n")
    f.write("  - Does hinge loss perform better than BCE?\n")
    f.write("  - Does larger Z_DIM (128) help or hurt?\n")
    f.write("  - Are there differences between MNIST and FashionMNIST?\n\n")
    
    # Add stability analysis
    f.write("**Stability Indicators from Your Experiments:**\n\n")
    for dataset_name in ['MNIST', 'FashionMNIST']:
        f.write(f"**{dataset_name}:**\n")
        stable_configs = []
        unstable_configs = []
        for exp_name, results in all_results['gan'].items():
            if results.get('dataset') != dataset_name:
                continue
            if results.get('mode_collapse_epochs'):
                unstable_configs.append((exp_name, results.get('loss_type'), results.get('z_dim')))
            else:
                stable_configs.append((exp_name, results.get('loss_type'), results.get('z_dim')))
        
        if stable_configs:
            f.write(f"- Stable configurations (no mode collapse): {len(stable_configs)}\n")
            for exp_name, loss_type, z_dim in stable_configs:
                f.write(f"  - {loss_type} loss, Z_DIM={z_dim}\n")
        if unstable_configs:
            f.write(f"- Unstable configurations (mode collapse detected): {len(unstable_configs)}\n")
            for exp_name, loss_type, z_dim in unstable_configs:
                f.write(f"  - {loss_type} loss, Z_DIM={z_dim}\n")
        f.write("\n")
    
    f.write("### 2. Evidence of mode collapse (if any)? What helped?\n\n")
    f.write("**Answer this question using:**\n")
    f.write("- The Mode Collapse Summary section above\n")
    f.write("- Visual inspection of generated samples (check for lack of diversity)\n")
    f.write("- Diversity scores: lower scores indicate less diversity\n\n")
    
    if mode_collapse_found:
        f.write("**Mode Collapse Detected:**\n")
        f.write("- Review which configurations experienced mode collapse\n")
        f.write("- Compare with stable configurations to identify what helped\n")
        f.write("- Common fixes: adjust learning rate, use different loss function, change Z_DIM\n\n")
    else:
        f.write("**No Mode Collapse Detected:**\n")
        f.write("- All experiments maintained good diversity\n")
        f.write("- This suggests your hyperparameters were well-chosen\n")
        f.write("- You can note: \"No mode collapse observed. Stable configurations included [list stable ones]\"\n\n")
    
    f.write("### 3. How did latent dim affect VAE reconstructions and samples?\n\n")
    f.write("**Analysis Guide:**\n")
    f.write("- Compare reconstruction losses across latent dimensions (8, 16, 32)\n")
    f.write("- Compare KL divergence: higher = more regularization\n")
    f.write("- Visual inspection: look at reconstruction quality and random sample quality\n")
    f.write("- FID scores: lower = better sample quality\n\n")
    
    f.write("**Latent Dimension Comparison:**\n\n")
    for dataset_name in ['MNIST', 'FashionMNIST']:
        f.write(f"**{dataset_name}:**\n")
        for latent_dim in [8, 16, 32]:
            exp_name = f"{dataset_name}_VAE_latent{latent_dim}"
            if exp_name in all_results['vae']:
                results = all_results['vae'][exp_name]
                f.write(f"- Latent dim {latent_dim}:\n")
                if results.get('losses'):
                    f.write(f"  - Final total loss: {results['losses'][-1]:.4f}\n")
                if results.get('recon_losses'):
                    f.write(f"  - Final recon loss: {results['recon_losses'][-1]:.4f}\n")
                if results.get('kl_losses'):
                    f.write(f"  - Final KL loss: {results['kl_losses'][-1]:.4f}\n")
                if 'fid' in results:
                    f.write(f"  - FID score: {results['fid']:.2f}\n")
        f.write("\n")
    
    f.write("**Expected patterns:**\n")
    f.write("- Lower latent dim (8): May have higher reconstruction error, but stronger regularization\n")
    f.write("- Higher latent dim (32): Better reconstruction, but may have less structured latent space\n")
    f.write("- Medium (16): Often a good balance\n\n")
    
    f.write("### 4. One idea to combine benefits of both models (e.g., VAE-GAN)\n\n")
    f.write("**Consider these approaches:**\n")
    f.write("- **VAE-GAN**: Use VAE encoder to provide structured latent space, GAN discriminator for sharpness\n")
    f.write("- **Adversarial VAE**: Add discriminator to VAE to improve sample quality\n")
    f.write("- **Latent GAN**: Train GAN in VAE's latent space instead of raw noise\n")
    f.write("- **Hybrid training**: Use VAE for reconstruction, GAN for generation\n\n")
    f.write("**Your observations:**\n")
    f.write("- GAN strengths: Sharp, high-quality samples (when stable)\n")
    f.write("- VAE strengths: Structured latent space, smooth interpolations, reconstruction capability\n")
    f.write("- Combination idea: [Write your idea here based on your results]\n\n")

print(f"\n{'='*60}")
print(f"Report saved to: {report_path}")
print(f"{'='*60}\n")

# Also create a summary text file
summary_path = f"{RESULTS_DIR}/experiment_summary.txt"
with open(summary_path, 'w') as f:
    f.write("EXPERIMENT SUMMARY\n")
    f.write("="*60 + "\n\n")
    f.write(f"Datasets: MNIST, FashionMNIST\n")
    f.write(f"GAN Experiments: {len(all_results['gan'])}\n")
    for exp_name in sorted(all_results['gan'].keys()):
        f.write(f"  - {exp_name}\n")
    f.write(f"\nVAE Experiments: {len(all_results['vae'])}\n")
    for exp_name in sorted(all_results['vae'].keys()):
        f.write(f"  - {exp_name}\n")
    f.write(f"\nAll outputs saved in: {RESULTS_DIR}/\n")

print(f"Summary saved to: {summary_path}")
print("\nAll experiments completed! Check the 'results/' directory for all outputs.")
